In [6]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = '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',
    model_name = "llama-3.3-70b-versatile"
)
response = llm.invoke("what is the shape of earth")
print(response.content)
    

The shape of the Earth is approximately an **oblate spheroid**, meaning that it is slightly flattened at the poles and bulging at the equator. This shape is also known as a **geoid**.

To be more precise, the Earth's shape can be described as:

1. **Equatorial radius**: approximately 6,378 kilometers (3,963 miles)
2. **Polar radius**: approximately 6,357 kilometers (3,949 miles)
3. **Equatorial circumference**: approximately 40,075 kilometers (24,901 miles)
4. **Polar circumference**: approximately 40,008 kilometers (24,860 miles)

The Earth's oblateness is caused by its rotation, which creates a centrifugal force that pushes the planet's mass away from the axis of rotation, resulting in a slightly flattened shape at the poles.

It's worth noting that the Earth is not a perfect sphere, but rather a complex and dynamic system with many irregularities and variations in its shape, such as:

* **Mountains and valleys**: the Earth's surface has many mountains, valleys, and other features th

In [8]:
!pip install chromadb

In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-i-itc/job/R-51569")

page_data = loader.load().pop().page_content
print(page_data)






















Software Engineer I , ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu





In [13]:
from langchain_core.prompts import PromptTemplate
import json
prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTIONS:
    The scrapped texxt is from the careers page of a website.
    your job is to extract  thr job postings and return them in JSON format containing the 
    following keys; 'role','experience','skills' and 'discription'.
    only retur the valid JSON.
    ###VALID JSON 
    NO PREAMBLE  STICTLY
    NO extra text other than josn file:
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)



```json
{
  "role": "Software Engineer I, ITC",
  "experience": "1+ years of professional experience as a Full Stack Software Engineer or in a similar role",
  "skills": [
    "Front-End Skills in React or similar technologies",
    "Server-side programming languages such as Node.js, Python or similar",
    "NoSQL databases",
    "API Development",
    "Cloud Technologies (AWS)",
    "Version Control (Git, GitHub)",
    "Testing Frameworks (unit testing, integration testing, TDD)"
  ],
  "description": "Design & Develop scalable web applications and services, Collaborate with product managers, designers, and other engineers, Front-End Development using modern JavaScript frameworks, Back-End Development using Node.js, Python or similar, Database Management, Testing & Quality Assurance"
}
```


In [14]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I, ITC',
 'experience': '1+ years of professional experience as a Full Stack Software Engineer or in a similar role',
 'skills': ['Front-End Skills in React or similar technologies',
  'Server-side programming languages such as Node.js, Python or similar',
  'NoSQL databases',
  'API Development',
  'Cloud Technologies (AWS)',
  'Version Control (Git, GitHub)',
  'Testing Frameworks (unit testing, integration testing, TDD)'],
 'description': 'Design & Develop scalable web applications and services, Collaborate with product managers, designers, and other engineers, Front-End Development using modern JavaScript frameworks, Back-End Development using Node.js, Python or similar, Database Management, Testing & Quality Assurance'}

In [15]:
type(json_res)


dict

In [16]:
import pandas as pd
df = pd.read_csv("tech_stack_exam_portfolio.csv")
df


,Tech Stack,Exam Portfolio Links
0,Python,link_to_python_exam_portfolio
1,C#,link_to_csharp_exam_portfolio
2,ASP.NET Core,link_to_aspnetcore_exam_portfolio
3,MERN,link_to_mern_exam_portfolio
4,Machine Learning,link_to_ml_exam_portfolio
5,Big Data,link_to_bigdata_exam_portfolio
6,Data Visualization,link_to_dataviz_exam_portfolio
7,NumPy,link_to_numpy_exam_portfolio
8,Pandas,link_to_pandas_exam_portfolio
9,Scikit-learn,link_to_sklearn_exam_portfolio


In [17]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectrostore')
collection = client.get_or_create_collection(name="portfolio")
if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents = row["Tech Stack"],
                       metadatas = {"links":row["Exam Portfolio Links"]},
                       ids = [str(uuid.uuid4())])

In [19]:
links = collection.query(query_texts=["Experience in python , expertice in Big Data"], n_results = 2).get("metadatas",[])
links

[[{'links': 'link_to_bigdata_exam_portfolio'},
  {'links': 'link_to_python_exam_portfolio'}]]

In [20]:
job = json_res
job["skills"]

['Front-End Skills in React or similar technologies',
 'Server-side programming languages such as Node.js, Python or similar',
 'NoSQL databases',
 'API Development',
 'Cloud Technologies (AWS)',
 'Version Control (Git, GitHub)',
 'Testing Frameworks (unit testing, integration testing, TDD)']

In [21]:
prompt_email = PromptTemplate.from_template(
    """
    ###JOB DESCRIPTION:
    {job_description}
    ###INSTRUCTION:
    You are jhon , a business developement executive at Crom.ai.Crom.ai is an AI $ Software consultant company 
    the seamless integrtion of business process through automated tools.
    Over out experience,we have empowred numerous enterprises with tailored solutions.
    process optimization,cost reduction,and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the 
    capability in fulfilling their needs.
    Also add the most relevant ones from the following links showcase croms's portfoli :{link_list}
    Remember you are john ,BDE at Crom.ai
    Do not provide preamble.
    ###EMAIL (NO PREAMBLE):
    """
)
chain_email = prompt_email|llm
res = chain_email.invoke({"job_description":str(job),"link_list":links})
print(res.content)

Subject: Expert Software Engineering Solutions for Seamless Integration

Dear Hiring Manager,

I came across the job description for a Software Engineer I, ITC, and I am excited to introduce Crom.ai as a potential partner in fulfilling your technical requirements. With over 1+ years of experience in delivering tailored software solutions, our team of experts is well-equipped to design and develop scalable web applications and services that meet your needs.

Our expertise in Front-End Development using modern JavaScript frameworks like React, combined with our proficiency in Server-side programming languages such as Node.js and Python, makes us an ideal fit for your project. We also have extensive experience in working with NoSQL databases, API Development, Cloud Technologies (AWS), and Version Control (Git, GitHub). Our testing frameworks, including unit testing, integration testing, and TDD, ensure that our solutions are thoroughly tested and of the highest quality.

At Crom.ai, we ha